# SI 618: Data Manipulation and Analysis
## 11a - Big Data I

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Introduction to Big Data

### **NOTE: conda install -y -c conda-forge mrjob**

[Distributed Computing I](assets/distcomp1.pdf)

### Generators

In [ ]:
def f():
    return 1

In [ ]:
def f123():
    yield 1
    yield 2
    yield 3

In [ ]:
def f123x(x):
    for i in range(0,x):
        yield i

In [ ]:
f()

In [ ]:
x = f()

In [ ]:
x

In [ ]:
f123()

In [ ]:
x = f123()

In [ ]:
x

In [ ]:
for thing in x:
    print(thing)

In [ ]:
x = f123x(10)

In [ ]:
x

In [ ]:
for thing in x:
    print(thing)

In [ ]:
x = f123x(10)

In [ ]:
for thing in x:
    print(thing)
    break;

In [ ]:
for thing in x:
    print(thing)
    break;

In [ ]:
next(x)

### Our first mrjob script

Note the use of the magic command ```%%file```.  You can use this to write the contents of a cell out to a file, which is what we need to do to use mrjob:

In [ ]:
%%file word_count.py
from mrjob.job import MRJob
import re

class MRWordFrequencyCount(MRJob):

  ### input: self, in_key, in_value
  def mapper(self, _, line):
    yield "chars", len(line)
    yield "words", len(line.split())
    yield "lines", 1

  ### input: self, in_key from mapper, in_value from mapper
  def reducer(self, key, values):
    yield key, sum(values)
if __name__ == "__main__":
    MRWordFrequencyCount.run()

Now we can run the script from the "command line", supplying a file to
analyze as the first argument.  We have supplied data/simpletext.txt, which contains the text from the slides, in today's zip file.

In [ ]:
!python word_count.py data/simpletext.txt

### <font color="magenta">Q1: Explain what the yield statements in the  above script do.  Provide a list of the first few iterations through the mapper() step would yield.

Insert your answer here.

### <font color="magenta">Q2.  Repeat the above cell using the Tale of Two Cities text file (data/totc.txt).  Report your findings below.

In [ ]:
# insert your code here

Insert your interpretation of the output here.

### Now let's look at a slightly more complicated example:

In [ ]:
%%file most_used_word.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRMostUsedWord.run()

In [1]:
!python most_used_word.py data/simpletext.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 2...
Creating temp directory /var/folders/1r/ht9dh7xn5j1ddwl8p7frd1282yw4hz/T/most_used_word.cteplovs.20190404.172306.080353
Running step 2 of 2...
job output is in /var/folders/1r/ht9dh7xn5j1ddwl8p7frd1282yw4hz/T/most_used_word.cteplovs.20190404.172306.080353/output
Streaming final output from /var/folders/1r/ht9dh7xn5j1ddwl8p7frd1282yw4hz/T/most_used_word.cteplovs.20190404.172306.080353/output...
3	"car"
Removing temp directory /var/folders/1r/ht9dh7xn5j1ddwl8p7frd1282yw4hz/T/most_used_word.cteplovs.20190404.172306.080353...


### <font color="magenta">Q3: Explain what the yield statements in the  above script do.  Provide a list of the first few iterations through the mapper() step would yield.

Insert your answer here.

### <font color="magenta">Q4: Run the above script on the Tale of Two Cities text file (data/totc.txt).  What answer do you get?</font>

In [1]:
# insert your command to run the script here

Insert your answer here.